<center>
<p style="text-align:center">
<img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
<br>
<br>
<a href="https://docs.arize.com/phoenix/">Docs</a>
|
<a href="https://github.com/Arize-ai/phoenix">GitHub</a>
|
<a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg?__hstc=259489365.a667dfafcfa0169c8aee4178d115dc81.1733501603539.1733501603539.1733501603539.1&__hssc=259489365.1.1733501603539&__hsfp=3822854628&submissionGuid=381a0676-8f38-437b-96f2-fc10875658df#/shared-invite/email">Community</a>
</p>
</center>
<h1 align="center">Comparing Prompt Optimization Techniques for an Agent Router</h1>

This tutorial will use Phoenix to compare the performance of different prompt optimization techniques on an Agent intent router.

You'll start by creating an experiment in Phoenix that can house the results of each of your resulting prompts. Next you'll use a series of prompt optimization techniques to improve the performance of an intent classification task for a customer support chatbot. Each technique will be applied to the same base prompt, and the results will be compared using Phoenix.

The techniques you'll use are:
- **Few Shot Examples**: Adding a few examples to the prompt to help the model understand the task.
- **Meta Prompting**: Prompting a model to generate a better prompt based on previous inputs, outputs, and expected outputs.
- **Prompt Gradients**: Using the gradient of the prompt to optimize individual components of the prompt using embeddings.
- **DSPy Prompt Tuning**: Using DSPy, an automated prompt tuning library, to optimize the prompt.

⚠️ This tutorial requires and OpenAI API key.

Let's get started!


### Setup Dependencies & Keys

In [ ]:
!pip install -q "arize-phoenix>=8.0.0" openinference-instrumentation-openai openai dspy

Next you need to connect to Phoenix. The code below will connect you to a Phoenix Cloud instance. You can also [connect to a self-hosted Phoenix instance](https://docs.arize.com/phoenix/deployment) if you'd prefer.

In [ ]:
import os
from getpass import getpass

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

### Load Dataset into Phoenix

Since we'll be running a series of experiments, we'll need a dataset of test cases that we can run each time. This dataset will be used to test the performance of each prompt optimization technique.

In [ ]:
import pandas as pd

user_intent_dataset = pd.read_csv("/Users/jgilhuly/Desktop/user-intent-test-cases.csv")
training_dataset = user_intent_dataset.sample(frac=0.2)
test_dataset = user_intent_dataset.drop(training_dataset.index)

In [ ]:
import uuid

import phoenix as px
from phoenix.client import Client as PhoenixClient

unique_id = uuid.uuid4()

# Upload the dataset to Phoenix
dataset = px.Client().upload_dataset(
    dataframe=test_dataset,
    input_keys=["input"],
    output_keys=["expected_intent"],
    dataset_name=f"user-intent-classification-{unique_id}",
)

Next, you can define a base template for the prompt. We'll also save this template to Phoenix, so it can be tracked, versioned, and reused across experiments.

In [ ]:
from openai import OpenAI
from openai.types.chat.completion_create_params import CompletionCreateParamsBase

from phoenix.client.types import PromptVersion

base_prompt = """
You are an intent classification system for a customer service chatbot. Your job is to identify the primary intent of customer messages and route them to the appropriate department.
Available Intents:

Account Management - Handle account-related issues
Billing Inquiry - Deal with billing questions
Technical Support - Provide technical support
Product Information - Answer product questions
Order Status - Track order status
Return/Exchange - Process returns
Complaint - Handle customer complaints
Feature Request - Collect feature suggestions
General Inquiry - Answer general questions
Cancellation - Process cancellations

Example:
User: "I need help with my login"
Intent: Account Management
"""

params = CompletionCreateParamsBase(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": base_prompt,
        },
        {"role": "user", "content": "{{question}}"},
    ],
)

prompt_identifier = "user-intent-classification"

prompt = PhoenixClient().prompts.create(
    name=prompt_identifier,
    prompt_description="A prompt for classifying user intent. Used as the first step of an agent.",
    version=PromptVersion.from_openai(params),
)

You should now see that prompt in Phoenix:

Next you'll need a task and evaluator for the experiment. A task is a function that will be run across each example in the dataset. The task is also the piece of your code that you'll change between each run of the experiment. To start off, the task is simply a call to GPT 3.5 Turbo with a basic prompt.

You'll also need an evaluator that will be used to test the performance of the task. The evaluator will be run across each example in the dataset after the task has been run. Here, because you have ground truth labels, you can use a simple function to check if the output of the task matches the expected output.

In [ ]:
client = OpenAI()


def test_prompt(input):
    resp = client.chat.completions.create(**prompt.format(variables={"question": input["input"]}))
    return resp.choices[0].message.content.strip()


def evaluate_response(output, expected):
    # Convert expected and output to lowercase
    expected_items = [
        item.strip().lower() for item in expected["expected_intent"].lower().split(",")
    ]
    output_lower = output.lower()

    # Handle comma-separated outputs
    if "," in output_lower:
        output_items = [item.strip() for item in output_lower.split(",")]
        return all(item in output_items for item in expected_items)

    # Simple string matching
    return all(item in output_lower for item in expected_items)

You can also instrument your code to send all models calls to Phoenix. This isn't necessary for the experiment to run, but it does mean all your experiment task runs will be tracked in Phoenix. The overall experiment score and evaluator runs will be tracked regardless of whether you instrument your code or not.

In [ ]:
from openinference.instrumentation.openai import OpenAIInstrumentor

from phoenix.otel import register

tracer_provider = register(project_name="agent-prompt-optimization")
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

Now you can run the initial experiment. This will be the base prompt that you'll be optimizing.

In [ ]:
import nest_asyncio

from phoenix.experiments import run_experiment

nest_asyncio.apply()

initial_experiment = run_experiment(
    dataset,
    task=test_prompt,
    evaluators=[evaluate_response],
    experiment_description="Initial base prompt",
    experiment_name="initial-prompt",
    experiment_metadata={"prompt": "prompt_id=" + prompt.id},
)

You should now see the initial experiment results in Phoenix:

# Prompt Optimization Technique #1: Few Shot Examples

One common prompt optimization technique is to use few shot examples to guide the model's behavior.

Here you can add few shot examples to the prompt to help improve performance. Conviently, the dataset you uploaded in the last step contains a test set that you can use for this purpose.

In [ ]:
# Extract a few examples from the dataset to use as few-shot examples
few_shot_examples = ""
for i in range(3):  # Using first 3 examples from dataset
    prompt_text = training_dataset.iloc[i]["input"]
    label = training_dataset.iloc[i]["expected_intent"]
    few_shot_examples += f"User input: {prompt_text}\nExpected intent: {label}\n\n"

Define a new prompt that includes the few shot examples. Prompts in Phoenix are automatically versioned, so saving the prompt with the same name will create a new version that can be used.

In [ ]:
few_shot_template = (
    base_prompt
    + """

Here are some examples of prompts and responses:

{examples}
"""
)

params = CompletionCreateParamsBase(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {"role": "system", "content": few_shot_template.format(examples=few_shot_examples)},
        {"role": "user", "content": "{{question}}"},
    ],
)

few_shot_prompt = PhoenixClient().prompts.create(
    name=prompt_identifier,
    prompt_description="Few shot prompt",
    version=PromptVersion.from_openai(params),
)

You'll notice you now have a new version of the prompt in Phoenix:

Define a new task with your new prompt:

In [ ]:
def test_prompt(input):
    client = OpenAI()
    prompt_vars = {"question": input["input"]}
    resp = client.chat.completions.create(**few_shot_prompt.format(variables=prompt_vars))
    return resp.choices[0].message.content.strip()

Now you can run another experiment with the new prompt. The dataset of test cases and the evaluator will be the same as the previous experiment.

In [ ]:
few_shot_experiment = run_experiment(
    dataset,
    task=test_prompt,
    evaluators=[evaluate_response],
    experiment_description="Prompt Optimization Technique #1: Few Shot Examples",
    experiment_name="few-shot-examples",
    experiment_metadata={"prompt": "prompt_id=" + few_shot_prompt.id},
)

# Prompt Optimization Technique #2: Meta Prompting

Meta prompting involves prompting a model to generate a better prompt, based on previous inputs, outputs, and expected outputs.

The experiment from round 1 serves as a great starting point for this technique, since it has each of those components.

In [ ]:
# Create a new column with the examples in a single string
training_dataset["example"] = training_dataset.apply(
    lambda row: f"User input: {row['input']}\nExpected intent: {row['expected_intent']}",
    axis=1,
)
training_dataset.head()

Now construct a new prompt that will be used to generate a new prompt.

In [ ]:
meta_prompt = """
You are an expert prompt engineer. You are given a prompt, and a list of examples with ground truth labels.

Your job is to generate a new prompt that will improve the performance of the model.

Here are the examples:

{ground_truth_examples}

Here is the original prompt:

{prompt}

Here is the new prompt:
"""

client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": meta_prompt.format(
                prompt=base_prompt, ground_truth_examples=training_dataset["example"].to_string()
            ),
        }
    ],
)
new_prompt = response.choices[0].message.content.strip()
print(new_prompt)

Now save that as a prompt in Phoenix:

In [ ]:
params = CompletionCreateParamsBase(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {"role": "system", "content": new_prompt},
        {"role": "user", "content": "{{question}}"},
    ],
)

meta_prompt_result = PhoenixClient().prompts.create(
    name=prompt_identifier,
    prompt_description="Meta prompt result",
    version=PromptVersion.from_openai(params),
)

### Run this new prompt through the same experiment
Redefine the task, using the new prompt.

In [ ]:
def test_prompt(input):
    client = OpenAI()
    resp = client.chat.completions.create(
        **meta_prompt_result.format(variables={"question": input["input"]})
    )
    return resp.choices[0].message.content.strip()

In [ ]:
meta_prompting_experiment = run_experiment(
    dataset,
    task=test_prompt,
    evaluators=[evaluate_response],
    experiment_description="Prompt Optimization Technique #2: Meta Prompting",
    experiment_name="meta-prompting",
    experiment_metadata={"prompt": "prompt_id=" + meta_prompt_result.id},
)

# Prompt Optimization Technique #3: Prompt Gradient Optimization

Prompt gradient optimization is a technique that uses the gradient of the prompt to optimize individual components of the prompt using embeddings. It involves:
1. Converting the prompt into an embedding.
2. Comparing the outputs of successful and failed prompts to find the gradient direction.
3. Moving in the gradient direction to optimize the prompt.

Here you'll define a function to get embeddings for prompts, and then use that function to calculate the gradient direction between successful and failed prompts.

In [ ]:
original_experiment_df = initial_experiment.as_dataframe()

original_experiment_df.head()

In [ ]:
successful_examples = original_experiment_df[
    original_experiment_df.apply(
        lambda row: evaluate_response(row["output"], row["expected"]), axis=1
    )
]
failed_examples = original_experiment_df[
    original_experiment_df.apply(
        lambda row: not evaluate_response(row["output"], row["expected"]), axis=1
    )
]

In [ ]:
import numpy as np


# First we'll define a function to get embeddings for prompts
def get_embedding(text):
    client = OpenAI()
    response = client.embeddings.create(model="text-embedding-ada-002", input=text)
    return response.data[0].embedding


# Function to calculate gradient direction between successful and failed prompts
def calculate_prompt_gradient(successful_prompts, failed_prompts):
    # Get embeddings for successful and failed prompts
    successful_embeddings = [get_embedding(p) for p in successful_prompts]
    failed_embeddings = [get_embedding(p) for p in failed_prompts]

    # Calculate average embeddings
    avg_successful = np.mean(successful_embeddings, axis=0)
    avg_failed = np.mean(failed_embeddings, axis=0)

    # Calculate gradient direction
    gradient = avg_successful - avg_failed
    return gradient / np.linalg.norm(gradient)


# Calculate the gradient direction
gradient = calculate_prompt_gradient(successful_examples[:5], failed_examples[:5])


# Function to optimize a prompt using the gradient
def optimize_prompt(base_prompt, gradient, step_size=0.1):
    # Get base embedding
    base_embedding = get_embedding(base_prompt)

    # Move in gradient direction
    optimized_embedding = base_embedding + step_size * gradient

    # Use GPT to convert the optimized embedding back to text
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are helping to optimize prompts. Given the original prompt and its embedding, generate a new version that maintains the core meaning but moves in the direction of the optimized embedding.",
            },
            {
                "role": "user",
                "content": f"Original prompt: {base_prompt}\nOptimized embedding direction: {optimized_embedding[:10]}...\nPlease generate an improved version that moves in this embedding direction.",
            },
        ],
    )
    return response.choices[0].message.content.strip()


# Test the gradient-based optimization
gradient_prompt = optimize_prompt(base_prompt, gradient)

In [ ]:
gradient_prompt

In [ ]:
params = CompletionCreateParamsBase(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": gradient_prompt,
        },
        {"role": "user", "content": "{{question}}"},
    ],
)

gradient_prompt = PhoenixClient().prompts.create(
    name=prompt_identifier,
    prompt_description="Gradient prompt result",
    version=PromptVersion.from_openai(params),
)

### Run experiment with gradient-optimized prompt
Redefine the task, using the new prompt.

In [ ]:
def test_gradient_prompt(input):
    client = OpenAI()
    resp = client.chat.completions.create(
        **gradient_prompt.format(variables={"question": input["input"]})
    )
    return resp.choices[0].message.content.strip()

In [ ]:
gradient_experiment = run_experiment(
    dataset,
    task=test_gradient_prompt,
    evaluators=[evaluate_response],
    experiment_description="Prompt Optimization Technique #3: Prompt Gradients",
    experiment_name="gradient-optimization",
    experiment_metadata={"prompt": "prompt_id=" + gradient_prompt.id},
)

# Prompt Optimization Technique #4: Prompt Tuning with DSPy

Finally, you can use an optimization library to optimize the prompt, like DSPy. [DSPy](https://github.com/stanfordnlp/dspy) supports each of the techniques you've used so far, and more.

Now you'll setup the DSPy language model and define a prompt classification task.

In [ ]:
# Import DSPy and set up the language model
from typing import Literal

import dspy

# Configure DSPy to use OpenAI
turbo = dspy.LM(model="gpt-3.5-turbo")
dspy.settings.configure(lm=turbo)


# Define the prompt classification task
class PromptClassifier(dspy.Signature):
    """Classify the intent of a user message."""

    input: str = dspy.InputField()
    intent: Literal[
        "Account Management",
        "No Intent",
        "Unclear Intent",
        "Escalation",
        "Billing Inquiry",
        "Technical Support",
        "Product Information",
        "Order Status",
        "Return/Exchange",
        "Complaint",
        "Feature Request",
        "General Inquiry",
        "Cancellation",
    ] = dspy.OutputField(
        desc="the intent of the user message, can choose one or more, comma separated"
    )


# Create the basic classifier
classifier = dspy.Predict(PromptClassifier)

Your classifier can now be used to make predictions as you would a normal LLM. It will expect a `prompt` input and will output a `label` prediction.

In [ ]:
classifier(prompt=user_intent_dataset.iloc[0].input)

However, DSPy really shines when it comes to optimizing prompts. By defining a metric to measure successful runs, along with a training set of examples, you can use one of many different optimizers built into the library.

In this case, you'll use the `MIPROv2` optimizer to find the best prompt for your task.

In [ ]:
def validate_classification(example, prediction, trace=None):
    return example["intent"] == prediction["intent"]


# Prepare training data from previous examples
train_data = []
for _, row in training_dataset.iterrows():
    example = dspy.Example(input=row["input"], label=row["expected_intent"]).with_inputs("input")
    train_data.append(example)

tp = dspy.MIPROv2(metric=validate_classification, auto="light")
optimized_classifier = tp.compile(classifier, trainset=train_data)

DSPy takes care of our prompts in this case, however you could still save the resulting prompt value in Phoenix:

In [ ]:
params = CompletionCreateParamsBase(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": optimized_classifier.signature.instructions,
        },  # if your meta prompt includes few shot examples, make sure to include them here
        {"role": "user", "content": "{{question}}"},
    ],
)

dspy_prompt = PhoenixClient().prompts.create(
    name=prompt_identifier,
    prompt_description="DSPy prompt result",
    version=PromptVersion.from_openai(params),
)

### Run experiment with DSPy-optimized classifier
Redefine the task, using the new prompt.

In [ ]:
# Create evaluation function using optimized classifier
def test_dspy_prompt(input):
    result = optimized_classifier(input=input["input"])
    return result.intent

In [ ]:
# Run experiment with DSPy-optimized classifier
dspy_experiment = run_experiment(
    dataset,
    task=test_dspy_prompt,
    evaluators=[evaluate_response],
    experiment_description="Prompt Optimization Technique #4: DSPy Prompt Tuning",
    experiment_name="dspy-optimization",
    experiment_metadata={"prompt": "prompt_id=" + dspy_prompt.id},
)

# You're done!

And just like that, you've run a series of prompt optimization techniques to improve the performance of an agent intent router, and compared the results using Phoenix.

You should have a set of experiments that looks like this:

From here, you can check out more [examples on Phoenix](https://docs.arize.com/phoenix/notebooks), and if you haven't already, [please give us a star on GitHub!](https://github.com/Arize-ai/phoenix) ⭐️